In [ ]:
# On installe open cv pour pouvoir par la suite classifier les visages, les yeux... etc
import cv2

In [ ]:
# On intalle le fichier de classification haarcascade_frontalface_default
face_cascade = cv2.CascadeClassifier('../input/haarcascade-face/haarcascade_frontalface_default.xml')

face_cascade

In [ ]:
# On boucle sur le fichier d'images
from os import listdir
from os.path import isfile, join

images_path = '../input/celeba-dataset/img_align_celeba/img_align_celeba/'

images = []

list_images = listdir(images_path)

# On ne charge que 13 000 images pour éviter une erreur de mémoire avec kaggle
for i in range(0,13000):
        f = list_images[i]
        if isfile(join(images_path, f)):
            images.append(images_path + f)

In [ ]:
images

In [ ]:
# On lit l'image d'entrée
modifImg = []
allCroped = []
allGray = []
for img in images:
    img = cv2.imread(img)
    # On convertit en niveaux de gris car cette méthode ne fonctionne que sur les images en niveaux de gris
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    allGray.append(gray)

    allFaces = [] 
    # On cherche à detecter les visages qui se ressemblent
    # le premier argument qu'on envoie est l'image en niveaux de gris
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    allFaces.append(faces)

    for id1, faces in enumerate(allFaces):
        for id2, (x, y, w, h) in enumerate(faces):

            # On recadre l'image
            crop_img = img[y:y+h, x:x+w]
            dsize = (100, 100)
            # On redimensionne l'image
            crop_img = cv2.resize(crop_img, dsize)
            allCroped.append(crop_img)
            print(len(allCroped))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Petit test pour voir ce que donne la dernière image stockée dans gray

plt.imshow(gray)
plt.show()

In [ ]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
list_img = []
for i,img_info in enumerate(allCroped):
    print (i, ' / ', len(allCroped))
    img_array = img_to_array(img_info)
    list_img.append(img_array)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
model = InceptionResNetV2(pooling='avg',include_top=False, weights='imagenet', input_shape=(100,100,3))
model.summary()

In [ ]:
import numpy as np
X = np.array(list_img)
X = preprocess_input(X)

In [ ]:
list_finale = []
for index in range(0, len(list_img)):
    print (index, ' / ', len(list_img))
    ret_vec = model.predict(X[index].reshape(1,100,100,3))
    list_finale.append({'vec':ret_vec.reshape(-1),'original':list_img[index], 'index':index})

In [ ]:
from keras.preprocessing.image import array_to_img

In [ ]:
to_find = 1

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt


plt.imshow(array_to_img(list_finale[to_find]['original']))
plt.show()

In [ ]:
from scipy import spatial
cosine_list = []
for index_image,xt in enumerate(list_finale):
    print (index_image, ' / ', len(list_finale))
    result = 1 - spatial.distance.cosine(list_finale[to_find]['vec'].reshape(-1), xt['vec'])
    cosine_list.append(dict({'res':result, 'i':list_finale[index_image]['index']}))
from operator import itemgetter
cosine_list.sort(key=itemgetter('res'), reverse=True)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=10,figsize=(20, 4))
plt.gray()
for indice, row in enumerate(ax):
    print (cosine_list[indice]['i'])
    row.imshow(array_to_img(list_finale[cosine_list[indice]['i']]['original']))

plt.show()

In [ ]:
cosine_list